# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [1]:
!pip install ultralytics -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.3/872.3 kB 15.7 MB/s eta 0:00:00


In [2]:
import cv2
from ultralytics import YOLO, solutions
from IPython.display import Video
import numpy as np
import shutil

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load the YOLO Model

In [55]:
model = YOLO('/content/drive/MyDrive/Tuwaiq/Week 6/6- Introduction to computer vision/5- Weekly Project (Sunday)/Dataset/YOLO_Model.pt')

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [84]:
vid = cv2.VideoCapture("/content/drive/MyDrive/Tuwaiq/Week 6/6- Introduction to computer vision/5- Weekly Project (Sunday)/Dataset/YOLO_Video.mp4")

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [85]:
w, h, fps = (int(vid.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
print("Video's height: ",h)
print("Video's width: ",w)
print("Video's FPS: ",fps)

Video's height:  1080
Video's width:  1920
Video's FPS:  29


## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [86]:
video_writer = cv2.VideoWriter("Yolo_Weekly_Project.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps,(w,h))

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [87]:
top_Line = np.array([
    (1160, 200),  # Top street exit
    (1050, 200)   # Top street exit
], dtype=np.int32)

left_Line = np.array([
    (525, 350),  # Left street exit
    (525, 550)   # Left street exit
], dtype=np.int32)

bottom_Line = np.array([
    (680, 1000),  # Bottom street exit
    (775, 1000)   # Bottom street exit
], dtype=np.int32)

right_Line = np.array([
    (1350, 725),  # Right street exit
    (1350, 545)   # Right street exit
], dtype=np.int32)

# lane_threshold_x_top = 1050 > 1160
lane_threshold_y_top = 200

lane_threshold_x_left = 525
# lane_threshold_y_left = 350 > 550

# lane_threshold_x_bottom =  680 > 775
lane_threshold_y_bottom = 1000

lane_threshold_x_right = 1350
# lane_threshold_y_right = 545 > 725



In [88]:
text_positions = {
    "top": (10, 50),
    "left": (10, 100),
    "right": (10, 150),
    "bottom": (10, 200)
}


font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 1
font_color = (255, 255, 255)
background_color = (0, 0, 0)


In [89]:
out = cv2.VideoWriter("YOLO_Practical_Exam.mp4", cv2.VideoWriter_fourcc(*'mp4v'), 20.0, (int(vid.get(3)), int(vid.get(4))))


while vid.isOpened():
    ret, frame = vid.read()
    if ret:
        detection = frame.copy()
        detection[:x1, :] = 0
        detection[x2:, :] = 0
        res = model.predict(detection)
        processed = res[0].plot(line_width=1)
        processed[:x1, :] = frame[:x1, :].copy()
        processed[x2:, :] = frame[x2:, :].copy()
        cv2.polylines(processed, [top_Line], isClosed=True, color=(0, 0, 255), thickness=5)
        cv2.polylines(processed, [left_Line], isClosed=True, color=(0, 0, 255), thickness=5)
        cv2.polylines(processed, [bottom_Line], isClosed=True, color=(0, 0, 255), thickness=5)
        cv2.polylines(processed, [right_Line], isClosed=True, color=(0, 0, 255), thickness=5)
        bounding_boxes = res[0].boxes


        vehicles_count = {"left": 0, "right": 0, "top": 0, "bottom": 0}
        for box in bounding_boxes.xyxy:
            center_x = (box[0] + box[2]) / 2
            center_y = (box[1] + box[3]) / 2
            if (center_x < lane_threshold_x_left) and (550 > center_y) and (center_y > 350):
                vehicles_count["left"] += 1
            elif (center_x > lane_threshold_x_right) and  (725 > center_y) and (center_y > 545)  :
                vehicles_count["right"] += 1
            elif (center_y < lane_threshold_y_top) and (1160> center_x) and (center_x > 1050):
                vehicles_count["top"] += 1
            elif (center_y > lane_threshold_y_bottom) and (775 > center_x) and (center_x > 680):
                vehicles_count["bottom"] += 1

        for lane, position in text_positions.items():
            cv2.rectangle(processed, (position[0] - 10, position[1] - 25),
                          (position[0] + 460, position[1] + 10), background_color, -1)
            cv2.putText(processed, f'Vehicles in {lane.capitalize()} Lane: {vehicles_count[lane]}',
                        position, font, font_scale, font_color, 2, cv2.LINE_AA)

        out.write(processed)
    else:
        break


vid.release()
out.release()



0: 384x640 1 bus, 46 cars, 1 van, 62.9ms
Speed: 4.4ms preprocess, 62.9ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bus, 46 cars, 2 vans, 59.2ms
Speed: 2.7ms preprocess, 59.2ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bus, 47 cars, 1 van, 56.0ms
Speed: 3.2ms preprocess, 56.0ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bus, 47 cars, 1 van, 49.2ms
Speed: 3.2ms preprocess, 49.2ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bus, 46 cars, 1 van, 49.2ms
Speed: 2.9ms preprocess, 49.2ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bus, 46 cars, 1 van, 40.6ms
Speed: 2.8ms preprocess, 40.6ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bus, 45 cars, 1 van, 39.5ms
Speed: 3.2ms preprocess, 39.5ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bus, 47 cars

In [91]:
video = cv2.VideoCapture('/content/YOLO_Practical_Exam.mp4')

fps = video.get(cv2.CAP_PROP_FPS)

min = 0
sec = 15
frame_id = int(fps*(min*60 + sec))
print('frame id =',frame_id)

video.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
ret, frame = video.read()

t_msec = 1000*(min*60 + sec)
video.set(cv2.CAP_PROP_POS_MSEC, t_msec)
ret, frame = video.read()

cv2.imwrite('myframe.png', frame)

# https://www.futurelearn.com/info/courses/introduction-to-image-analysis-for-plant-phenotyping/0/steps/305359

frame id = 300


True

## Save and Submit Your Work

In [82]:
# Video("YOLO_Practical_Exam.mp4", embed=True, width=960)

In [ ]:
source_path = '/content/YOLO_Practical_Exam.mp4'
destination_path = '/content/drive/MyDrive/Tuwaiq'

shutil.move(source_path, destination_path)

Submit both the notebook and the output video